In [14]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TweedieRegressor
import wrangle
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from pydataset import data
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
df = pd.read_csv('final_df.csv',index_col=[0])
columns_list = df.select_dtypes(exclude=['object']).columns.to_list()

list_to_remove = ['conf_champ','avg_att_air_yards','avg_air_yard_diff','time_to_throw','air_yds_to_sticks','won_superbowl',
'avg_comp_air_yds','superbowl','year']

for i in list_to_remove:
    if i in columns_list:
        columns_list.remove(i)

        
target = columns_list.pop(len(columns_list) -1 )

In [9]:
X_train

,aggressive,lon_comp_air_dist,pass_att,total_yds,td,int,pass_rating,completion_pct,exp_completion_pct,completion_pct_abv_exp,first_round,div_round
name,,,,,,,,,,,,
Tom Brady,15.2,50.9,613.0,4057.0,24.0,8.0,88.0,60.8,64.0,-3.1,1.0,0.0
Matt Ryan,16.3,52.0,560.0,3968.0,20.0,12.0,90.4,67.0,66.0,0.9,0.0,0.0
Trevor Siemian,20.4,57.3,486.0,3401.0,18.0,10.0,84.6,59.5,57.5,2.0,0.0,0.0
Matt Barkley,21.8,47.7,216.0,1611.0,8.0,14.0,68.3,59.7,58.7,1.0,0.0,0.0
Jalen Hurts,14.2,50.7,148.0,1061.0,6.0,4.0,77.6,52.0,55.5,-3.4,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
Drew Stanton,20.1,47.0,159.0,894.0,6.0,5.0,66.4,49.7,59.1,-9.4,0.0,0.0
Davis Mills,15.2,52.9,479.0,3118.0,17.0,15.0,78.8,61.0,64.2,-3.2,0.0,0.0
Andy Dalton,15.0,48.3,333.0,2170.0,14.0,8.0,87.3,64.9,64.9,0.0,0.0,0.0


In [20]:
X = df[columns_list]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=123)
parameters_lars = {
    'alpha':range(1,21),
    'fit_intercept':[True, False],
    'verbose': [True, False]
}

parameters_linear = {'n_jobs':range(1,21)
                }

parameters_glm = {
    'link':['auto', 'identity', 'log'],
    'alpha':range(1,21),
}

lars = LassoLars(random_state =123)
Linear_regression1 = LinearRegression()
glm = TweedieRegressor(power=2, alpha=0)


the_parameters = [parameters_lars, parameters_linear, parameters_glm]


models = ['lars','Linear_regression1','glm']
master_df = pd.DataFrame()
for number, tree in enumerate([lars, Linear_regression1, glm]):
    grid = GridSearchCV(tree, the_parameters[number], cv=5, scoring = 'neg_root_mean_squared_error')
    grid.fit(X_train, y_train)
    
    for p, score in zip(grid.cv_results_['params'], grid.cv_results_['mean_test_score']):
        p['score'] = abs(score)
        p['model'] = models[number]
    new_df = pd.DataFrame(pd.DataFrame(grid.cv_results_['params']).sort_values('score', ascending=True))
    master_df = pd.concat([master_df, new_df])

In [21]:
master_df

,alpha,fit_intercept,verbose,score,model,n_jobs,link
79,20.0,False,False,6.128393,lars,NaN,NaN
78,20.0,False,True,6.128393,lars,NaN,NaN
75,19.0,False,False,6.129814,lars,NaN,NaN
74,19.0,False,True,6.129814,lars,NaN,NaN
70,18.0,False,True,6.131291,lars,NaN,NaN
71,18.0,False,False,6.131291,lars,NaN,NaN
66,17.0,False,True,6.132822,lars,NaN,NaN
67,17.0,False,False,6.132822,lars,NaN,NaN
76,20.0,True,True,6.133877,lars,NaN,NaN
77,20.0,True,False,6.133877,lars,NaN,NaN
